In [1]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)
import re
import ast
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")

## reading the data

In [435]:
df_new = pd.read_excel('zillow_full.xlsx')

## selecting some columns

In [488]:
a=['countyId','zipcode', 'zestimate','lotSize','resoFacts_cityRegion',
  'rentZestimate','bedrooms', 'bathrooms', 'livingAreaValue',  
       'propertyTaxRate',  'listing_sub_type_is_newHome',
       'resoFacts_hasFireplace', 'resoFacts_parking',
       'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate', 
    'city', 'county','propertyTypeDimension' , 'description','parentRegion_name',
     'resoFacts_daysOnZillow', 'favoriteCount', 'resoFacts_hasAttachedProperty',
    'resoFacts_hasGarage', 'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa' , 'resoFacts_hasView',
     'resoFacts_pricePerSquareFoot','resoFacts_yearBuilt','ssid','schools'
   ]

In [489]:
df=df_new[a]

In [490]:
df.shape

(1325, 32)

In [491]:
df.isnull().sum()

countyId                                0
zipcode                                 0
zestimate                             252
lotSize                               120
resoFacts_cityRegion                    0
rentZestimate                         164
bedrooms                               49
bathrooms                              24
livingAreaValue                        23
propertyTaxRate                         2
listing_sub_type_is_newHome             0
resoFacts_hasFireplace                223
resoFacts_parking                       0
mortgageRates_thirtyYearFixedRate       7
mortgageRates_fifteenYearFixedRate      9
mortgageRates_arm5Rate                  9
city                                    0
county                                  0
propertyTypeDimension                   0
description                             3
parentRegion_name                       2
resoFacts_daysOnZillow                  2
favoriteCount                           4
resoFacts_hasAttachedProperty     

In [492]:
df = df.dropna()

In [493]:
df.shape

(841, 32)

# Feature Engineering- Creating 6 features out of school

In [494]:
schools= list(df['schools'])

In [495]:
list_schools= [[]] * len(schools)

In [496]:
len(list_schools)

841

In [497]:
i=0
for x in schools:
    list_schools[i]=[0,0,0,0,0,0]
    x = str(x).replace('\\','"')
    l=re.findall("[{][\\a-zA-z0-9:',\-/ \.=?&]+[}]", x)
    for y in l:
        y=ast.literal_eval(y)
        if y['level']== 'Elementary':
            if y['rating']!=None:
                if list_schools[i][1]<y['rating']:
                    list_schools[i][1]=y['rating']
                    list_schools[i][0]=y['distance']
                
        if y['level']== 'Middle':
            if y['rating']!=None:
                if list_schools[i][3]<y['rating']:
                    list_schools[i][3]=y['rating']
                    list_schools[i][2]=y['distance']
        if y['level']=='High':
            if y['rating']!=None:
                if list_schools[i][5]<y['rating']:
                    list_schools[i][5]=y['rating']
                    list_schools[i][4]=y['distance']
    
    i+=1
    
    

In [498]:
list_schools[0]

[0.3, 2, 0.9, 2, 0.5, 7]

In [499]:
for i in range(len(list_schools)):
    list_schools[i] = [None if item <=0 else item for item in list_schools[i]]

In [500]:
df_3 = pd.DataFrame(list_schools, columns = ['Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance', 'Middle_Ratings', 'High_Distance', 'High_Ratings'])

In [501]:
df_3.head()

,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings
0,0.30000,2.00000,0.90000,2.00000,0.50000,7.00000
1,1.60000,5.00000,2.60000,3.00000,nan,nan
2,0.10000,4.00000,2.70000,4.00000,0.10000,10.00000
3,0.90000,5.00000,3.00000,3.00000,0.80000,2.00000
4,0.20000,7.00000,0.80000,6.00000,1.10000,8.00000


In [502]:
df_3['Elementary_Distance']=np.where(np.isnan(df_3['Elementary_Distance']),df_3['Elementary_Distance'].median(),df_3['Elementary_Distance']) 
df_3['Middle_Ratings']=np.where(np.isnan(df_3['Middle_Ratings']),df_3['Middle_Ratings'].mode(),df_3['Middle_Ratings']) 
df_3['Middle_Distance']=np.where(np.isnan(df_3['Middle_Distance']),df_3['Middle_Distance'].median(),df_3['Middle_Distance']) 
df_3['High_Ratings']=np.where(np.isnan(df_3['High_Ratings']),df_3['High_Ratings'].mode(),df_3['High_Ratings']) 
df_3['High_Distance']=np.where(np.isnan(df_3['High_Distance']),df_3['High_Distance'].median(),df_3['High_Distance']) 


In [503]:
from sklearn.impute import KNNImputer   # missing values imputation

imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_3.iloc[:,0:]=imputer.fit_transform(df_3.iloc[:,0:]) #replace df with imputed missing values

In [504]:
df_3.head()

,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings
0,0.30000,2.00000,0.90000,2.00000,0.50000,7.00000
1,1.60000,5.00000,2.60000,3.00000,1.00000,7.00000
2,0.10000,4.00000,2.70000,4.00000,0.10000,10.00000
3,0.90000,5.00000,3.00000,3.00000,0.80000,2.00000
4,0.20000,7.00000,0.80000,6.00000,1.10000,8.00000


In [505]:
df_3.shape

(841, 6)

In [506]:
df.shape

(841, 32)

In [507]:
df.drop('schools', axis=1, inplace= True)

In [508]:
df.shape

(841, 31)

In [509]:
df=df.reset_index()

In [510]:
df=pd.concat([df,df_3], axis=1)

In [511]:
df.shape  # this df contains school columns also

(841, 38)

# Feature Engineering- Creating New feature out of feature("description")

In [512]:
sentiment_des = []
from textblob import TextBlob
for text in df['description'].values:
        t= TextBlob(text)
        sentiment_des.append(t.sentiment[0])
df['desc_sentiment']=sentiment_des

In [513]:
df['desc_sentiment']=sentiment_des

In [514]:
df.drop(['description'], axis=1,inplace=True)

In [515]:
df.shape 

(841, 38)

In [516]:
df.corr()

,index,countyId,zipcode,zestimate,lotSize,rentZestimate,bedrooms,bathrooms,livingAreaValue,propertyTaxRate,listing_sub_type_is_newHome,resoFacts_hasFireplace,resoFacts_parking,mortgageRates_thirtyYearFixedRate,mortgageRates_fifteenYearFixedRate,mortgageRates_arm5Rate,resoFacts_daysOnZillow,favoriteCount,resoFacts_hasAttachedProperty,resoFacts_hasGarage,resoFacts_hasPetsAllowed,resoFacts_hasSpa,resoFacts_hasView,resoFacts_pricePerSquareFoot,resoFacts_yearBuilt,ssid,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings,desc_sentiment
index,1.00000,-0.16707,0.05231,-0.04938,0.04074,-0.05534,0.02774,-0.04493,-0.03149,0.16419,0.05563,0.04400,0.02314,-0.11089,-0.07792,0.09968,-0.02747,0.04891,-0.08079,0.01250,-0.03025,0.00087,0.02427,-0.03309,0.03522,0.01149,0.00533,-0.00034,0.02488,0.09240,0.02288,-0.01707,-0.01310
countyId,-0.16707,1.00000,0.05771,0.18334,-0.05840,0.17344,-0.13702,-0.08021,-0.10964,-0.60249,-0.07689,0.01524,-0.32134,0.15688,0.32746,-0.32170,-0.26826,-0.15792,0.30847,-0.37361,0.02560,0.11370,0.01321,0.47167,-0.03775,-0.18402,-0.08286,0.18287,0.05104,0.03231,-0.11981,-0.01738,-0.00464
zipcode,0.05231,0.05771,1.00000,-0.20958,-0.02422,-0.26703,0.13498,0.02412,-0.01513,0.40100,-0.04788,0.30450,-0.15656,-0.03595,-0.03595,0.02269,-0.05305,0.06689,-0.47042,-0.34594,-0.27011,0.17893,-0.06342,-0.30876,0.11572,0.10564,0.04455,-0.05937,-0.10388,0.01735,0.12562,0.03846,0.04282
zestimate,-0.04938,0.18334,-0.20958,1.00000,0.13820,0.88808,0.45404,0.61609,0.75768,-0.21848,0.02241,0.16950,0.25143,0.28930,0.55896,-0.53766,0.26460,0.03196,0.08092,0.06879,0.02611,0.02229,0.19298,0.46896,-0.05141,-0.03945,0.11877,0.35710,0.08950,0.27293,0.09887,0.22804,0.02493
lotSize,0.04074,-0.05840,-0.02422,0.13820,1.00000,0.12478,0.20872,0.11230,0.21981,0.02694,-0.00855,0.01595,0.08206,0.01542,0.02956,-0.03135,0.21756,0.00515,0.00046,-0.03204,-0.01027,-0.01868,-0.00872,-0.01665,0.01269,0.25268,0.29254,0.01489,0.16344,0.03771,0.22443,0.03118,-0.01809
rentZestimate,-0.05534,0.17344,-0.26703,0.88808,0.12478,1.00000,0.39347,0.59141,0.71877,-0.31285,-0.00141,0.08460,0.27415,0.20675,0.44233,-0.41951,0.23163,0.02321,0.18816,0.12056,0.06935,-0.00379,0.19848,0.32992,-0.12572,-0.06271,0.08274,0.25324,0.06975,0.16750,0.01949,0.10929,0.01556
bedrooms,0.02774,-0.13702,0.13498,0.45404,0.20872,0.39347,1.00000,0.66438,0.69062,0.19332,-0.01655,0.29311,0.23891,0.22161,0.43934,-0.42884,0.25311,0.12189,-0.27300,0.07245,-0.23432,0.06975,0.06792,-0.19039,0.07784,0.14992,0.08760,0.13807,-0.01335,0.17720,0.16501,0.21940,0.01887
bathrooms,-0.04493,-0.08021,0.02412,0.61609,0.11230,0.59141,0.66438,1.00000,0.81442,0.06365,0.09995,0.19266,0.35246,0.23356,0.41439,-0.39629,0.32351,0.07619,-0.11769,0.16506,-0.07530,0.09453,0.16463,-0.12712,0.04185,0.04144,0.16462,0.24608,0.12989,0.19896,0.23333,0.21615,-0.01107
livingAreaValue,-0.03149,-0.10964,-0.01513,0.75768,0.21981,0.71877,0.69062,0.81442,1.00000,0.04425,0.04543,0.21848,0.40203,0.24999,0.44103,-0.42579,0.42382,0.09809,-0.10987,0.20023,-0.03558,0.08428,0.19747,-0.11421,0.04234,0.04692,0.20039,0.24393,0.13392,0.20846,0.20736,0.20699,-0.00080
propertyTaxRate,0.16419,-0.60249,0.40100,-0.21848,0.02694,-0.31285,0.19332,0.06365,0.04425,1.00000,0.02114,0.21662,0.08743,-0.09594,-0.17999,0.17285,0.18337,0.14285,-0.53327,-0.02639,-0.27140,0.03600,-0.08863,-0.40119,0.18921,0.20434,0.11513,-0.12115,-0.10353,0.01033,0.22692,0.06514,0.05025


# Train-Test Split

In [517]:
df.select_dtypes(include='number').columns

Index(['index', 'countyId', 'zipcode', 'zestimate', 'lotSize', 'rentZestimate',
       'bedrooms', 'bathrooms', 'livingAreaValue', 'propertyTaxRate',
       'listing_sub_type_is_newHome', 'resoFacts_hasFireplace',
       'resoFacts_parking', 'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate',
       'resoFacts_daysOnZillow', 'favoriteCount',
       'resoFacts_hasAttachedProperty', 'resoFacts_hasGarage',
       'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa', 'resoFacts_hasView',
       'resoFacts_pricePerSquareFoot', 'resoFacts_yearBuilt', 'ssid',
       'Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance',
       'Middle_Ratings', 'High_Distance', 'High_Ratings', 'desc_sentiment'],
      dtype='object')

In [518]:
y = df.zestimate

In [519]:
x= df.drop('zestimate', axis=1)

In [520]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2, random_state=123)

In [521]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(672, 37)
(672,)
(169, 37)
(169,)


# One Hot Encoding

In [579]:
cat_columns=['city', 'county','propertyTypeDimension']

In [580]:
from scipy.sparse import coo_matrix

X_train_cat_processed = coo_matrix((672, 0))
X_test_cat_processed = coo_matrix((169, 0))

In [581]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

from sklearn import preprocessing
vectorizer= preprocessing.OneHotEncoder(handle_unknown = 'ignore')
for x in cat_columns:
    
    
# 1. INSTANTIATE
    
    

        
    one_hot_train = vectorizer.fit_transform(X_train[x].values.reshape(-1,1))
    one_hot_test = vectorizer.transform(X_test[x].values.reshape(-1,1))
    vectorizer.get_feature_names()
    
    X_train_cat_processed= hstack((X_train_cat_processed,one_hot_train)).tocsr()
    X_test_cat_processed= hstack((X_test_cat_processed,one_hot_test)).tocsr()



In [582]:
print(X_train_cat_processed.shape)
print(X_test_cat_processed.shape)

(672, 22)
(169, 22)


# Numeric Data Preparation

In [584]:
from scipy.sparse import coo_matrix

X_train_num_processed = coo_matrix((672,0))
X_test_num_processed = coo_matrix((169,0))

In [593]:
num_cols= [ 'lotSize', 
       'bedrooms', 'bathrooms', 'livingAreaValue', 
        'resoFacts_hasFireplace',
       'resoFacts_hasGarage',
       'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa', 'resoFacts_hasView',
       'resoFacts_pricePerSquareFoot', 'resoFacts_yearBuilt', 
       'Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance',
       'Middle_Ratings', 'High_Distance', 'High_Ratings']

In [594]:
from scipy.sparse import hstack

for x in num_cols:
    

    s_train = (X_train[x].values.reshape(-1, 1))

    s_test = (X_test[x].values.reshape(-1, 1))

    
    X_train_num_processed= hstack((X_train_num_processed,s_train)).tocsr()
    X_test_num_processed= hstack((X_test_num_processed,s_test)).tocsr()


In [595]:
print(X_train_num_processed.shape)
print(X_test_num_processed.shape)

(672, 37)
(169, 37)


# Linear Regression

In [596]:
from scipy.sparse import hstack
X_train_processed = hstack((X_train_cat_processed,X_train_num_processed )).tocsr()
X_test_processed = hstack(( X_test_cat_processed,X_test_num_processed)).tocsr()
print(X_test_processed.shape)
print(X_train_processed.shape)

(169, 59)
(672, 59)


In [597]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train_processed, Y_train)

LinearRegression()

In [598]:
Y_pred = regressor.predict(X_test_processed)

In [599]:
df_out = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
df_out.head(8)

,Actual,Predicted
656,1566500.00000,1774043.40124
275,3369400.00000,3310102.71305
327,780500.00000,705373.61843
285,792158.00000,776794.77732
280,951000.00000,1178171.02468
753,2249775.00000,1939328.99008
202,2185300.00000,2138251.94313
145,1837400.00000,1743212.19978


In [604]:
from sklearn.metrics import r2_score
r2_score =r2_score(Y_test, Y_pred)
rows , col =X_train_processed.shape
adj_R2 = 1- ((1-r2_score) * (rows-1)/(rows-col-1)) #Adj R2 = 1-(1-R2)*(n-1)/(n-p-1)
print('R2 is:', r2_score.round(4))
print('Adjusted R2 is:', adj_R2.round(3))


R2 is: 0.9293
Adjusted R2 is: 0.922
